# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [1]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

# **2. 데이터 전처리**



In [2]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

In [3]:
#train_dataset = tf.data.Dataset.from_tensor_slices((x_train_after, y_train))
#test_dataset = tf.data.Dataset.from_tensor_slices((x_test1_after, y_test1))
num_examples = len(x_train)
print(x_train.shape)
print(y_train.shape)
print(num_examples)

train_dataset = tf.data.Dataset.from_tensor_slices(({'image': x_train, 'label': y_train}))
test_dataset = tf.data.Dataset.from_tensor_slices(({'image': x_test1, 'label': y_test1}))
test_dataset2 = tf.data.Dataset.from_tensor_slices(({'image': x_test2, 'label': y_test2}))
print(train_dataset)
print(test_dataset)
print(test_dataset2)

(50000, 32, 32, 3)
(50000, 10)
50000
<TensorSliceDataset shapes: {image: (32, 32, 3), label: (10,)}, types: {image: tf.uint8, label: tf.float32}>
<TensorSliceDataset shapes: {image: (32, 32, 3), label: (10,)}, types: {image: tf.uint8, label: tf.float32}>
<TensorSliceDataset shapes: {image: (32, 32, 3), label: (10,)}, types: {image: tf.uint8, label: tf.float32}>


### Preprocessing, Data Augmentation

In [4]:
# Preprocessing helper functions

# Create data pipelines for training and testing:
BATCH_SIZE = 512
SCHEDULE_LENGTH = 512
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE
RESIZE_TO = 160
STEPS_PER_EPOCH = 10
CROP_TO = 128

def cast_to_tuple(features):
  print(features)
  return (features['image'], features['label'])
  
def preprocess_train(features):
  #print(features.shape)
  # Apply random crops and horizontal flips for all tasks 
  # except those for which cropping or flipping destroys the label semantics
  # (e.g. predict orientation of an object)
  features['image'] = tf.image.random_flip_left_right(features['image'])
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.image.random_crop(features['image'], [CROP_TO, CROP_TO, 3])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

def preprocess_test(features):
  features['image'] = tf.image.resize(features['image'], [RESIZE_TO, RESIZE_TO])
  features['image'] = tf.cast(features['image'], tf.float32) / 255.0
  return features

pipeline_train = (train_dataset
                  .shuffle(10000)
                  .repeat(int(SCHEDULE_LENGTH * BATCH_SIZE / num_examples * STEPS_PER_EPOCH) + 1 + 50)  # repeat dataset_size / num_steps
                  .map(preprocess_train, num_parallel_calls=8)
                  .batch(BATCH_SIZE)
                  .map(cast_to_tuple)  # for keras model.fit
                  .prefetch(2))

pipeline_test = (test_dataset.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))
pipeline_test2 = (test_dataset2.map(preprocess_test, num_parallel_calls=1)
                  .map(cast_to_tuple)  # for keras model.fit
                  .batch(BATCH_SIZE)
                  .prefetch(2))

{'image': <tf.Tensor 'args_0:0' shape=(None, 128, 128, 3) dtype=float32>, 'label': <tf.Tensor 'args_1:0' shape=(None, 10) dtype=float32>}
{'image': <tf.Tensor 'args_0:0' shape=(160, 160, 3) dtype=float32>, 'label': <tf.Tensor 'args_1:0' shape=(10,) dtype=float32>}
{'image': <tf.Tensor 'args_0:0' shape=(160, 160, 3) dtype=float32>, 'label': <tf.Tensor 'args_1:0' shape=(10,) dtype=float32>}


# **3. 모델 생성**

### 3-1 BigTransfer 전이학습 준비

In [5]:
import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib
import tensorflow_hub as hub

In [6]:
model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
module = hub.KerasLayer(model_url)

### Bit model 

prelogit head를 우리가 원하는 custom dataset에 맞게 수정한다.
따라서 전체를 학습할 필요가 없이 전이 학습이 가능하다

In [8]:
# Add new head to the BiT model

class MyBiTModel(tf.keras.Model):
  """BiT with a new head."""

  def __init__(self, num_classes, module):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.bit_model = module
  
  def call(self, images):
    # No need to cut head off since we are using feature extractor model
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = MyBiTModel(num_classes=num_classes, module=module)

In [9]:
# Define optimiser and loss

lr = 0.003 * BATCH_SIZE / 512 

# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[200,300,400], 
                                                                   values=[lr, lr*0.1, lr*0.001, lr*0.0001])
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [10]:
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

# Fine-tune model
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    #epochs= int(SCHEDULE_LENGTH / STEPS_PER_EPOCH), # TODO: replace with `epochs=10` here to shorten fine-tuning for tutorial if you wish
    epochs = 5,
    validation_data=pipeline_test # here we are only using 
                                   # this data to evaluate our performance
)

Epoch 1/5
10/10 [==============================] - 22s 2s/step - loss: 0.9148 - accuracy: 0.6859 - val_loss: 0.2805 - val_accuracy: 0.9085
Epoch 2/5
10/10 [==============================] - 44s 4s/step - loss: 0.3855 - accuracy: 0.8805 - val_loss: 0.2850 - val_accuracy: 0.9140
Epoch 3/5
10/10 [==============================] - 70s 7s/step - loss: 0.3761 - accuracy: 0.8807 - val_loss: 0.2597 - val_accuracy: 0.9195
Epoch 4/5
10/10 [==============================] - 70s 7s/step - loss: 0.3568 - accuracy: 0.8902 - val_loss: 0.2771 - val_accuracy: 0.9164
Epoch 5/5
10/10 [==============================] - 71s 7s/step - loss: 0.3373 - accuracy: 0.8977 - val_loss: 0.2628 - val_accuracy: 0.9175


# **4. 모델 저장**

In [11]:
save_path = './content/'
team_name = 'team17'

# 트레이닝된 전체 모델을 저장합니다.
model.save_weights(save_path +  'model_entire_'+ team_name + '.h5')
#model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [12]:
# save_path = './content/'
team_name = 'team17'

model.load_weights(save_path + 'model_entire_' + team_name + '.h5')

#model.evaluate(x_test1_after, y_test1)
#model.evaluate(x_test2_after, y_test2)
#모델을 pipelining 해서 평가함


model.evaluate(pipeline_test)
model.evaluate(pipeline_test2)

4/4 [==============================] - 23s 6s/step - loss: 0.5188 - accuracy: 0.8320


[0.5187661051750183, 0.8320000171661377]